## 一、RNN 无框架实现
符号沿用上一章，
n:  批量大小
d:  输入数据维度
h:  隐藏层单元数
q:  输出数据维度

In [9]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

batch_size, num_steps = 32, 35  # 32，35 没有太多含义，总得有个值
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)
len(vocab)

28

### 1.独热编码
F.one_hot() 将一个张量中的每一个数值变成一个[0,...,1,0,0]的独热编码，其长度=字典的长度
显然，对输入的张量升了 1 维。

In [10]:
F.one_hot(torch.tensor([0, 2]), len(vocab))

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]])

 one_hot函数将小批量数据（二维张量：（批量大小，时间步数））转换成三维张量， 张量的最后一个维度等于词表大小（len(vocab)）。

In [11]:
# 转置矩阵的目的是实现连续性（不太理解，原话见https://www.bilibili.com/video/BV1kq4y1H7sw/?spm_id_from=333.999.0.0&vd_source=46af8e25a61e33c10249cebb913791e3  05:57）
X = torch.arange(10).reshape((2, 5))
F.one_hot(X.T, 28).shape

torch.Size([5, 2, 28])

### 2.初始化参数
输入和输出具有相同的维度，即词表的大小。

In [12]:
# 词表大小，隐藏层单元数，GPU地址
def get_params(vocab_size, num_hiddens, device):
    # 此处设置 d = q = vocab_size
    num_inputs = num_outputs = vocab_size

    # 初始化函数,创建大小为 shape,值服从正态随机分布的 torch
    def normal(shape):
        return torch.randn(size=shape, device=device) * 0.01

    # 隐藏层参数
    W_xh = normal((num_inputs, num_hiddens))        # d * h
    W_hh = normal((num_hiddens, num_hiddens))       # h * h
    b_h = torch.zeros(num_hiddens, device=device)   # 1 * h
    # 输出层参数
    W_hq = normal((num_hiddens, num_outputs))       # h * q
    b_q = torch.zeros(num_outputs, device=device)   # 1 * q
    # 附加梯度
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params

### 3.初始化隐藏状态
为了定义循环神经网络模型， 我们首先需要一个`init_rnn_state`函数在初始化时返回隐藏状态。
这个函数的返回是一个张量，张量全用 0 填充，
形状为（批量大小，隐藏单元数）。

In [13]:
def init_rnn_state(batch_size, num_hiddens, device):
    # 此时的 RNN 的确只需要1个参数,但后续的 LSTM 需要2个，在此处写成长度为 2 的 tuple 是为了保持代码形式一致
    return (torch.zeros((batch_size, num_hiddens), device=device), )

### 4. 循环神经网络模型

In [14]:
# inputs的形状：(时间步数量，批量大小，词表大小)
def rnn(inputs, state, params):
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state  # 初始化隐藏矩阵
    outputs = []
    # X的形状：(批量大小，词表大小)
    for X in inputs:
        H = torch.tanh(torch.mm(X, W_xh) + torch.mm(H, W_hh) + b_h)  # 每一步 H 都考虑上一步 H
        Y = torch.mm(H, W_hq) + b_q
        outputs.append(Y)   # outputs 保存每个时刻的输出

    # output 竖直堆叠，列数不变，行数增加
    return torch.cat(outputs, dim=0), (H,)

使用一个类来包装这些函数

In [15]:
class RNNModelScratch: #@save
    """从零开始实现的循环神经网络模型"""
    def __init__(self, vocab_size, num_hiddens, device,
                 get_params, init_state, forward_fn):
        self.vocab_size, self.num_hiddens = vocab_size, num_hiddens
        self.params = get_params(vocab_size, num_hiddens, device)
        self.init_state, self.forward_fn = init_state, forward_fn

    def __call__(self, X, state):
        X = F.one_hot(X.T, self.vocab_size).type(torch.float32)
        return self.forward_fn(X, state, self.params)

    def begin_state(self, batch_size, device):
        return self.init_state(batch_size, self.num_hiddens, device)

In [16]:
num_hiddens = 512
net = RNNModelScratch(len(vocab), num_hiddens, d2l.try_gpu(), get_params,
                      init_rnn_state, rnn)
state = net.begin_state(X.shape[0], d2l.try_gpu())
Y, new_state = net(X.to(d2l.try_gpu()), state)
Y.shape, len(new_state), new_state[0].shape

(torch.Size([10, 28]), 1, torch.Size([2, 512]))

[10,28]的尺寸表示一个 batch 内的2条 * 5个单词，词表长度为28；值表示这 10 个词预测的下一个词（词典中的某一个）
new_state 是一个长度=1的tuple
new_state[0].shape 表示(批量大小,隐藏层单元数)

### 4. 预测

In [ ]:
# prefix:       给定句子开头，用于生成后续的句子
# num_preds:    要求生成多少个词
# vocab:        把预测值（数字）转换为文本（单词）
def predict_ch8(prefix, num_preds, net, vocab, device):  #@save
    """在prefix后面生成新字符"""
    state = net.begin_state(batch_size=1, device=device) # batch_size=1 表示对一个字符串做预测
    outputs = [vocab[prefix[0]]]    # 把单词在词表中的 index 存到 outputs
    get_input = lambda: torch.tensor([outputs[-1]], device=device).reshape((1, 1))# 把上一次最近预测到的词作为输入

    for y in prefix[1:]:  # 预热期
        _, state = net(get_input(), state)
        outputs.append(vocab[y])
    for _ in range(num_preds):  # 预测num_preds步
        y, state = net(get_input(), state)
        outputs.append(int(y.argmax(dim=1).reshape(1)))
    return ''.join([vocab.idx_to_token[i] for i in outputs])

### 5. 梯度裁剪
### 6. 训练ddd